# TRY1

In [1]:
# combined_rul_pipeline.py

import os
import sys
import json
import time
import h5py
import joblib
import torch
import pandas as pd
import numpy as np

from datetime import datetime
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from tab_transformer_pytorch import TabTransformer
from torch.optim import Adam
from torch.nn import MSELoss
from sklearn.metrics import mean_squared_error, mean_absolute_error

# === Hard-coded paths ===
WINDOW_CSV     = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\Code\super_same_norm.csv"
SPEC_CSV       = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\data\train_specifications.csv"
ENCODER_PATH   = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\spec_encoder.joblib"
H5_PATH        = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\data_windows.h5"
ARTIFACT_ROOT  = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\artifacts"
VALIDATION_CSV = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\Code\validation_super_same_norm.csv"
VALIDATION_H5  = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\validation_data.h5"

SENSOR_FEATURES = [
    '171_0', '666_0', '427_0', '837_0', '167_0', '167_1', '167_2', '167_3', '167_4',
    '167_5', '167_6', '167_7', '167_8', '167_9', '309_0', '272_0', '272_1', '272_2',
    '272_3', '272_4', '272_5', '272_6', '272_7', '272_8', '272_9', '835_0', '370_0',
    '291_0', '291_1', '291_2', '291_3', '291_4', '291_5', '291_6', '291_7', '291_8',
    '291_9', '291_10', '158_0', '158_1', '158_2', '158_3', '158_4', '158_5', '158_6',
    '158_7', '158_8', '158_9', '100_0', '459_0', '459_1', '459_2', '459_3', '459_4',
    '459_5', '459_6', '459_7', '459_8', '459_9', '459_10', '459_11', '459_12', '459_13',
    '459_14', '459_15', '459_16', '459_17', '459_18', '459_19', '397_0', '397_1', '397_2',
    '397_3', '397_4', '397_5', '397_6', '397_7', '397_8', '397_9', '397_10', '397_11',
    '397_12', '397_13', '397_14', '397_15', '397_16', '397_17', '397_18', '397_19',
    '397_20', '397_21', '397_22', '397_23', '397_24', '397_25', '397_26', '397_27',
    '397_28', '397_29', '397_30', '397_31', '397_32', '397_33', '397_34', '397_35'
]

# === Utils ===
def create_X_y(csv_path=WINDOW_CSV, sensor_features=None, context=70, verbose=True):
    df = pd.read_csv(csv_path)
    X,y,vids = [],[],[]
    for vid,grp in df.groupby("vehicle_id"):
        data = grp[sensor_features].values
        rul  = grp["RUL"].values
        if len(data)<context:
            if verbose: print(f"Skipping {vid}, len<{context}")
            continue
        for i in range(len(data)-context+1):
            X.append(data[i:i+context])
            y.append(rul[i+context-1])
            vids.append(vid)
    X=np.stack(X); y=np.array(y); vids=np.array(vids)
    if verbose: print(f"Windows: {len(X)}, shape={X.shape[1:]}")
    spec_df = pd.read_csv(SPEC_CSV)
    spec_cols=[f"Spec_{i}" for i in range(8)]
    enc=OrdinalEncoder()
    spec_df[spec_cols]=enc.fit_transform(spec_df[spec_cols])
    specs = (
        pd.DataFrame({"vehicle_id":vids})
          .merge(spec_df[["vehicle_id"]+spec_cols],on="vehicle_id")
    )[spec_cols].values.astype(int)
    joblib.dump(enc, ENCODER_PATH)
    if verbose: print(f"Saved encoder→{ENCODER_PATH}")
    return X,y,vids,specs

def save_to_h5(X,y,vids,specs,h5_path=H5_PATH):
    with h5py.File(h5_path,"w") as f:
        f.create_dataset("X_windows",data=X,compression="gzip")
        f.create_dataset("y_labels", data=y,compression="gzip")
        f.create_dataset("window_vids",data=vids,compression="gzip")
        f.create_dataset("specs_per_window",data=specs,compression="gzip")
    print(f"Saved H5→{h5_path}")

def load_from_h5(h5_path=H5_PATH):
    with h5py.File(h5_path,"r") as f:
        X=f["X_windows"][:]
        y=f["y_labels"][:]
        vids=f["window_vids"][:]
        specs=f["specs_per_window"][:]
    return X,y,vids,specs

class RULCombinedDataset(Dataset):
    def __init__(self,X,specs,y):
        self.X=X; self.specs=specs; self.y=y.reshape(-1,1)
    def __len__(self): return len(self.y)
    def __getitem__(self,i):
        return (
            torch.from_numpy(self.specs[i]).long(),
            torch.from_numpy(self.X[i]).float(),
            torch.from_numpy(self.y[i]).float()
        )

def make_artifact_folder(model_name,pvt):
    ts=datetime.now().strftime("%Y%m%d_%H%M%S")
    suffix="DP" if pvt else "NDP"
    folder=f"{model_name}-{suffix}-{ts}"
    path=os.path.join(ARTIFACT_ROOT,folder)
    os.makedirs(path,exist_ok=True)
    return path

# === Models ===
class TimeSeriesEmbedder(nn.Module):
    def __init__(self,num_features,d_model=128,n_heads=8,num_layers=2,dropout=0.1):
        super().__init__()
        self.input_proj=nn.Linear(num_features,d_model)
        enc_layer=nn.TransformerEncoderLayer(
            d_model=d_model,nhead=n_heads,dropout=dropout,batch_first=True
        )
        self.encoder=nn.TransformerEncoder(enc_layer,num_layers=num_layers)
    def forward(self,x):
        x=self.input_proj(x)
        x=self.encoder(x)
        return x[:,-1,:]

class CombinedRULModel(nn.Module):
    def __init__(self,num_sensor_features,context_length,categories,continuous_dim,cont_mean_std=None):
        super().__init__()
        self.tf=TimeSeriesEmbedder(num_sensor_features,continuous_dim)
        if cont_mean_std is None:
            cont_mean_std=torch.stack([torch.zeros(continuous_dim),torch.ones(continuous_dim)],dim=1)
        self.tab=TabTransformer(
            categories=categories,
            num_continuous=continuous_dim,
            dim=continuous_dim,
            dim_out=1,
            depth=6,
            heads=8,
            attn_dropout=0.1,
            ff_dropout=0.1,
            mlp_hidden_mults=(4,2),
            mlp_act=nn.ReLU(),
            continuous_mean_std=cont_mean_std
        )
    def forward(self,x_cat,x_ts):
        cont=self.tf(x_ts)
        return self.tab(x_cat,cont)

# === Services ===
def get_criterion(): return MSELoss()
def get_optimizer(model,lr=1e-3): return Adam(model.parameters(),lr=lr)



In [2]:
import torch
torch.backends.cuda.preferred_linalg_library("magma")


<_LinalgBackend.Magma: 2>

In [4]:
def train_spectral_dp_batch(model, criterion, optimizer, x_cat, x_ts, yb,
                            clipping_norm, noise_scale, delta):
    optimizer.zero_grad()
    output = model(x_cat, x_ts)
    tloss = criterion(output, yb)
    tloss.backward()

    grads = [p.grad.detach() for p in model.parameters() if p.grad is not None]

    valid_grads = []
    for g in grads:
        if torch.isnan(g).any() or torch.isinf(g).any():
            print("Invalid gradient tensor encountered. Skipping batch.")
            return
        valid_grads.append(g)

    # Flatten and stack
    grads_flat = torch.cat([g.reshape(-1) for g in valid_grads])
    num_params = grads_flat.shape[0]
    G = grads_flat.reshape(num_params, 1)

    if torch.isnan(G).any() or torch.isinf(G).any():
        print("NaNs/Infs in flattened gradient matrix G. Skipping batch.")
        return

    # Try GPU SVD, fallback to CPU if fails
    try:
        U, S, Vh = torch.linalg.svd(G, full_matrices=False)
    except RuntimeError as e:
        print("SVD failed on GPU, trying CPU fallback. Error:", e)
        try:
            G_cpu = G.cpu()
            U_cpu, S_cpu, Vh_cpu = torch.linalg.svd(G_cpu, full_matrices=False)
            # Move back to GPU
            U = U_cpu.to(G.device)
            S = S_cpu.to(G.device)
            Vh = Vh_cpu.to(G.device)
        except RuntimeError as e2:
            print("SVD also failed on CPU. Skipping batch. Error:", e2)
            return

    # Clip and noised singular values
    S_clipped = torch.clamp(S, max=clipping_norm)
    noise = torch.normal(mean=0, std=noise_scale, size=S.shape, device=S.device)
    S_noisy = S_clipped + noise

    G_noisy = (U @ torch.diag(S_noisy) @ Vh)

    # Unflatten and write back noisy gradients
    offset = 0
    for p in model.parameters():
        if p.grad is None:
            continue
        numel = p.grad.numel()
        p.grad.data = G_noisy[offset:offset+numel].reshape(p.grad.shape)
        offset += numel

    optimizer.step()


In [5]:
# import torch
# torch.backends.cuda.preferred_linalg_library("magma")


In [6]:

def train(use_h5=True, pvt=False):
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BATCH_SIZE, NUM_EPOCHS, LR = 1, 5, 1e-3
    LR_PAT, ES_PAT, LR_F = 5, 11, 0.5

    # DP params
    DP_CLIP, DP_NOISE, DP_DELTA = 1.0, 1.0, 1e-5

    # data load
    if use_h5 and os.path.exists(H5_PATH):
        X, y, _, specs = load_from_h5()
    else:
        X, y, _, specs = create_X_y(sensor_features=SENSOR_FEATURES)
        save_to_h5(X, y, _, specs)

    encoder = joblib.load(ENCODER_PATH)
    cat_sizes = tuple(len(c) for c in encoder.categories_)

    Xtr, Xv, str_, sv, yr, yv = train_test_split(X, specs, y, test_size=0.2, random_state=42)
    train_loader = DataLoader(RULCombinedDataset(Xtr, str_, yr), batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(RULCombinedDataset(Xv, sv, yv), batch_size=BATCH_SIZE, shuffle=False)

    # setup artifacts, metadata
    art = make_artifact_folder("CombinedRULModel",pvt)
    logp, metap, ckpt = [os.path.join(art, fn) for fn in ("train_val_log.txt","metadata.json","checkpoint.pth")]
    meta = {"model_name":"CombinedRULModel","num_sensor_features":X.shape[2],
            "context_length":X.shape[1],"continuous_dim":128,
            "categories":list(cat_sizes),"batch_size":BATCH_SIZE,
            "learning_rate":LR,"num_epochs":NUM_EPOCHS,"pvt":pvt}
    with open(metap,"w") as f: json.dump(meta,f,indent=4)

    model = CombinedRULModel(X.shape[2],X.shape[1],cat_sizes,128).to(DEVICE)
    criterion, optimizer = MSELoss(), Adam(model.parameters(),lr=LR)
    scheduler = StepLR(optimizer,step_size=1,gamma=LR_F)

    best, noimp = float('inf'), 0
    with open(logp,"w") as f: f.write("epoch,train_loss,val_loss,epoch_time,lr,notes\n")

    start = time.perf_counter()
    for ep in range(1, NUM_EPOCHS+1):
        ep_start = time.perf_counter()
        lr_cur = optimizer.param_groups[0]['lr']

        # training
        model.train(); train_loss=0
        for x_cat, x_ts, yb in train_loader:
            x_cat, x_ts, yb = x_cat.to(DEVICE), x_ts.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            if pvt:
                train_spectral_dp_batch(model, criterion, optimizer, x_cat, x_ts, yb,
                                        DP_CLIP, DP_NOISE, DP_DELTA)
            else:
                loss = criterion(model(x_cat, x_ts), yb)
                loss.backward(); optimizer.step(); train_loss += loss.item()*yb.size(0)
        if not pvt:
            train_loss /= len(train_loader.dataset)

        # validation
        model.eval(); val_loss=0
        with torch.no_grad():
            for x_cat, x_ts, yb in val_loader:
                x_cat, x_ts, yb = x_cat.to(DEVICE), x_ts.to(DEVICE), yb.to(DEVICE)
                loss = criterion(model(x_cat, x_ts), yb)
                val_loss += loss.item()*yb.size(0)
        val_loss /= len(val_loader.dataset)

        # logging & sched
        elapsed = time.perf_counter()-ep_start
        h,m,s = map(int,[elapsed//3600,(elapsed%3600)//60,elapsed%60])
        notes=""
        if val_loss<best:
            best,val_notes=val_loss,f"Model saved at epoch {ep}"
            torch.save(model.state_dict(), ckpt)
            notes=val_notes
        else:
            noimp+=1
            if noimp%LR_PAT==0: scheduler.step(); notes+=" LR stepped"
            if noimp>=ES_PAT: notes+=" Early stopping"

        with open(logp,"a") as f:
            f.write(f"{ep},{train_loss:.6f},{val_loss:.6f},{h:02d}:{m:02d}:{s:02d},{lr_cur:.6g},{notes}\n")
        print(f"Epoch {ep:02d} Train {train_loss:.4f} Val {val_loss:.4f} Time {h:02d}:{m:02d}:{s:02d} LR {lr_cur:.2e} {notes}")
        if noimp>=ES_PAT: break

    total = time.perf_counter()-start
    h,m,s = map(int,[total//3600,(total%3600)//60,total%60]); tt=f"{h:02d}:{m:02d}:{s:02d}"
    with open(metap,"r+") as f:
        d=json.load(f); d["total_training_time"]=tt; f.seek(0); json.dump(d,f,indent=4); f.truncate()
    print(f"Done. Best val MSE {best:.4f}. Total time {tt}")





# I AM STARTING HERE :)
if __name__=="__main__":
   
    train(use_h5=True, pvt=True)

KeyboardInterrupt: 

In [ ]:
!taskkill /PID 9412 /F

In [ ]:
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
Cell In[4], line 98
     95 # I AM STARTING HERE :)
     96 if __name__=="__main__":
---> 98     train(use_h5=True, pvt=True)

Cell In[4], line 47
     45 model.train(); train_loss=0
     46 for x_cat, x_ts, yb in train_loader:
---> 47     x_cat, x_ts, yb = x_cat.to(DEVICE), x_ts.to(DEVICE), yb.to(DEVICE)
     48     optimizer.zero_grad()
     49     if pvt:

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

# TRY2

In [9]:

# import os
# import sys
# import json
# import time
# import h5py
# import joblib
# import torch
# import pandas as pd
# import numpy as np

# from datetime import datetime
# from torch import nn
# from torch.utils.data import Dataset, DataLoader
# from torch.optim.lr_scheduler import StepLR
# from sklearn.preprocessing import OrdinalEncoder
# from sklearn.model_selection import train_test_split
# from tab_transformer_pytorch import TabTransformer
# from torch.optim import Adam
# from torch.nn import MSELoss
# from sklearn.metrics import mean_squared_error, mean_absolute_error

# # === Hard-coded paths ===
# WINDOW_CSV     = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\Code\super_same_norm.csv"
# SPEC_CSV       = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\data\train_specifications.csv"
# ENCODER_PATH   = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\spec_encoder.joblib"
# H5_PATH        = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\data_windows.h5"
# ARTIFACT_ROOT  = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\artifacts"
# VALIDATION_CSV = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\Code\validation_super_same_norm.csv"
# VALIDATION_H5  = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\validation_data.h5"

# SENSOR_FEATURES = [
#     '171_0', '666_0', '427_0', '837_0', '167_0', '167_1', '167_2', '167_3', '167_4',
#     '167_5', '167_6', '167_7', '167_8', '167_9', '309_0', '272_0', '272_1', '272_2',
#     '272_3', '272_4', '272_5', '272_6', '272_7', '272_8', '272_9', '835_0', '370_0',
#     '291_0', '291_1', '291_2', '291_3', '291_4', '291_5', '291_6', '291_7', '291_8',
#     '291_9', '291_10', '158_0', '158_1', '158_2', '158_3', '158_4', '158_5', '158_6',
#     '158_7', '158_8', '158_9', '100_0', '459_0', '459_1', '459_2', '459_3', '459_4',
#     '459_5', '459_6', '459_7', '459_8', '459_9', '459_10', '459_11', '459_12', '459_13',
#     '459_14', '459_15', '459_16', '459_17', '459_18', '459_19', '397_0', '397_1', '397_2',
#     '397_3', '397_4', '397_5', '397_6', '397_7', '397_8', '397_9', '397_10', '397_11',
#     '397_12', '397_13', '397_14', '397_15', '397_16', '397_17', '397_18', '397_19',
#     '397_20', '397_21', '397_22', '397_23', '397_24', '397_25', '397_26', '397_27',
#     '397_28', '397_29', '397_30', '397_31', '397_32', '397_33', '397_34', '397_35'
# ]

# # === Utils ===
# def create_X_y(csv_path=WINDOW_CSV, sensor_features=SENSOR_FEATURES, context=70, verbose=True):
#     df = pd.read_csv(csv_path)
#     X, y, vids = [], [], []
#     for vid, grp in df.groupby("vehicle_id"):
#         data = grp[sensor_features].values
#         rul = grp["RUL"].values
#         if len(data) < context:
#             if verbose:
#                 print(f"Skipping {vid}, len<{context}")
#             continue
#         for i in range(len(data) - context + 1):
#             X.append(data[i:i+context])
#             y.append(rul[i+context-1])
#             vids.append(vid)
#     X = np.stack(X)
#     y = np.array(y)
#     vids = np.array(vids)
#     if verbose:
#         print(f"Windows: {len(X)}, shape={X.shape[1:]}")

#     spec_df = pd.read_csv(SPEC_CSV)
#     spec_cols = [f"Spec_{i}" for i in range(8)]
#     enc = OrdinalEncoder()
#     spec_df[spec_cols] = enc.fit_transform(spec_df[spec_cols])
#     specs = (
#         pd.DataFrame({"vehicle_id": vids})
#           .merge(spec_df[["vehicle_id"] + spec_cols], on="vehicle_id")
#     )[spec_cols].values.astype(int)

#     joblib.dump(enc, ENCODER_PATH)
#     if verbose:
#         print(f"Saved encoder → {ENCODER_PATH}")

#     return X, y, vids, specs


# def save_to_h5(X, y, vids, specs, h5_path=H5_PATH):
#     with h5py.File(h5_path, "w") as f:
#         f.create_dataset("X_windows", data=X, compression="gzip")
#         f.create_dataset("y_labels", data=y, compression="gzip")
#         f.create_dataset("window_vids", data=vids, compression="gzip")
#         f.create_dataset("specs_per_window", data=specs, compression="gzip")
#     print(f"Saved H5 → {h5_path}")


# def load_from_h5(h5_path=H5_PATH):
#     with h5py.File(h5_path, "r") as f:
#         X = f["X_windows"][:]
#         y = f["y_labels"][:]
#         vids = f["window_vids"][:]
#         specs = f["specs_per_window"][:]
#     return X, y, vids, specs


# class RULCombinedDataset(Dataset):
#     def __init__(self, X, specs, y):
#         self.X = X
#         self.specs = specs
#         self.y = y.reshape(-1, 1)

#     def __len__(self):
#         return len(self.y)

#     def __getitem__(self, i):
#         return (
#             torch.from_numpy(self.specs[i]).long(),
#             torch.from_numpy(self.X[i]).float(),
#             torch.from_numpy(self.y[i]).float()
#         )


# def make_artifact_folder(model_name):
#     ts = datetime.now().strftime("%Y%m%d_%H%M%S")
#     suffix = "NDP"
#     folder = f"{model_name}-{suffix}-{ts}"
#     path = os.path.join(ARTIFACT_ROOT, folder)
#     os.makedirs(path, exist_ok=True)
#     return path

# # === Models ===
# class TimeSeriesEmbedder(nn.Module):
#     def __init__(self, num_features, d_model=128, n_heads=8, num_layers=2, dropout=0.1):
#         super().__init__()
#         self.input_proj = nn.Linear(num_features, d_model)
#         enc_layer = nn.TransformerEncoderLayer(
#             d_model=d_model, nhead=n_heads, dropout=dropout, batch_first=True
#         )
#         self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)

#     def forward(self, x):
#         x = self.input_proj(x)
#         x = self.encoder(x)
#         return x[:, -1, :]


# class CombinedRULModel(nn.Module):
#     def __init__(self, num_sensor_features, context_length, categories, continuous_dim, cont_mean_std=None):
#         super().__init__()
#         self.tf = TimeSeriesEmbedder(num_sensor_features, continuous_dim)
#         if cont_mean_std is None:
#             cont_mean_std = torch.stack([torch.zeros(continuous_dim), torch.ones(continuous_dim)], dim=1)
#         self.tab = TabTransformer(
#             categories=categories,
#             num_continuous=continuous_dim,
#             dim=continuous_dim,
#             dim_out=1,
#             depth=6,
#             heads=8,
#             attn_dropout=0.1,
#             ff_dropout=0.1,
#             mlp_hidden_mults=(4,2),
#             mlp_act=nn.ReLU(),
#             continuous_mean_std=cont_mean_std
#         )

#     def forward(self, x_cat, x_ts):
#         cont = self.tf(x_ts)
#         return self.tab(x_cat, cont)


# # === Services ===
# def get_criterion():
#     return MSELoss()


# def get_optimizer(model, lr=1e-3):
#     return Adam(model.parameters(), lr=lr)


# # === Training function ===
# def train(use_h5=False):
#     DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     BATCH_SIZE = 256
#     NUM_EPOCHS = 50
#     LR = 1e-3
#     LR_PAT = 5
#     ES_PAT = 11
#     LR_F = 0.5

#     # data
#     if use_h5 and os.path.exists(H5_PATH):
#         X, y, _, specs = load_from_h5()
#     else:
#         X, y, _, specs = create_X_y(sensor_features=SENSOR_FEATURES)
#         save_to_h5(X, y, _, specs)

#     encoder = joblib.load(ENCODER_PATH)
#     cat_sizes = tuple(len(c) for c in encoder.categories_)

#     Xtr, Xv, str_, sv, yr, yv = train_test_split(X, specs, y, test_size=0.2, random_state=42)
#     tl = DataLoader(RULCombinedDataset(Xtr, str_, yr), batch_size=BATCH_SIZE, shuffle=True)
#     vl = DataLoader(RULCombinedDataset(Xv, sv, yv), batch_size=BATCH_SIZE, shuffle=False)

#     # artifacts
#     art = make_artifact_folder("CombinedRULModel")
#     logp = os.path.join(art, "train_val_log.txt")
#     metap = os.path.join(art, "metadata.json")
#     ckpt = os.path.join(art, "checkpoint.pth")

#     # metadata init
#     meta = {
#         "model_name": "CombinedRULModel",
#         "num_sensor_features": X.shape[2],
#         "context_length": X.shape[1],
#         "continuous_dim": 128,
#         "categories": list(cat_sizes),
#         "batch_size": BATCH_SIZE,
#         "learning_rate": LR,
#         "num_epochs": NUM_EPOCHS
#     }
#     with open(metap, "w") as f:
#         json.dump(meta, f, indent=4)

#     # model setup
#     model = CombinedRULModel(X.shape[2], X.shape[1], cat_sizes, 128).to(DEVICE)
#     crit = get_criterion()
#     opt = get_optimizer(model, lr=LR)
#     sched = StepLR(opt, step_size=1, gamma=LR_F)

#     best = float('inf')
#     noimp = 0

#     with open(logp, "w") as f:
#         f.write("epoch,train_loss,val_loss,epoch_time,lr,notes
# ")

#     start_all = time.perf_counter()
#     for ep in range(1, NUM_EPOCHS + 1):
#         ep_start = time.perf_counter()
#         lr_cur = opt.param_groups[0]['lr']

#         # train
#         model.train()
#         tloss = 0
#         for xc, xt, yb in tl:
#             xc, xt, yb = xc.to(DEVICE), xt.to(DEVICE), yb.to(DEVICE)
#             opt.zero_grad()
#             l = crit(model(xc, xt), yb)
#             l.backward()
#             opt.step()
#             tloss += l.item() * yb.size(0)
#         tloss /= len(tl.dataset)

#         # validation
#         model.eval()
#         vloss = 0
#         with torch.no_grad():
#             for xc, xt, yb in vl:
#                 xc, xt, yb = xc.to(DEVICE), xt.to(DEVICE), yb.to(DEVICE)
#                 l = crit(model(xc, xt), yb)
#                 vloss += l.item() * yb.size(0)
#         vloss /= len(vl.dataset)

#         # timing and checkpoint logic
#         elapsed = time.perf_counter() - ep_start
#         h, m, s = map(int, [elapsed//3600, (elapsed%3600)//60, elapsed%60])
#         et = f"{h:02d}:{m:02d}:{s:02d}"
#         notes = ""
#         if vloss < best:
#             best = vloss
#             noimp = 0
#             torch.save(model.state_dict(), ckpt)
#             notes = f"Model saved at epoch {ep}"
#         else:
#             noimp += 1
#             if noimp % LR_PAT == 0:
#                 sched.step()
#                 notes += " LR stepped"
#             if noimp >= ES_PAT:
#                 notes += " Early stopping"

#         with open(logp, "a") as f:
#             f.write(f"{ep},{tloss:.6f},{vloss:.6f},{et},{lr_cur:.6g},{notes.strip()}
# ")
#         print(f"Epoch {ep:02d} Train {tloss:.4f} Val {vloss:.4f} Time {et} LR {lr_cur:.2e} {notes}")

#         if noimp >= ES_PAT:
#             print("Early stop")
#             break

#     total = time.perf_counter() - start_all
#     h, m, s = map(int, [total//3600, (total%3600)//60, total%60])
#     tt = f"{h:02d}:{m:02d}:{s:02d}"

#     # update metadata
#     with open(metap, "r+") as f:
#         d = json.load(f)
#         d["total_training_time"] = tt
#         f.seek(0)
#         json.dump(d, f, indent=4)
#         f.truncate()

#     print(f"Done. Best val MSE {best:.4f}. Total time {tt}")

# # === Entry Point ===
# if __name__ == "__main__":
#     train(use_h5=True)


In [1]:
import os
import sys
import json
import time
import h5py
import joblib
import torch
import pandas as pd
import numpy as np

from datetime import datetime
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from tab_transformer_pytorch import TabTransformer
from torch.optim import Adam
from torch.nn import MSELoss
from sklearn.metrics import mean_squared_error, mean_absolute_error

# === Hard-coded paths ===
WINDOW_CSV     = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\Code\super_same_norm.csv"
SPEC_CSV       = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\data\train_specifications.csv"
ENCODER_PATH   = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\spec_encoder.joblib"
H5_PATH        = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script\data_windows.h5"
ARTIFACT_ROOT  = r"C:\Users\ASUS\Desktop\SCANIA\2024-34-2\2024-34-2\Important_script2\artifacts"

# === Utils ===
def create_X_y(csv_path=WINDOW_CSV, sensor_features=None, context=70, verbose=True):
    df = pd.read_csv(csv_path)
    X, y, vids = [], [], []
    for vid, grp in df.groupby("vehicle_id"):
        data = grp[sensor_features].values
        rul = grp["RUL"].values
        if len(data) < context:
            if verbose:
                print(f"Skipping {vid}, len<{context}")
            continue
        for i in range(len(data) - context + 1):
            X.append(data[i:i+context])
            y.append(rul[i+context-1])
            vids.append(vid)
    X = np.stack(X)
    y = np.array(y)
    vids = np.array(vids)
    if verbose:
        print(f"Windows: {len(X)}, shape={X.shape[1:]}")

    spec_df = pd.read_csv(SPEC_CSV)
    spec_cols = [f"Spec_{i}" for i in range(8)]
    enc = OrdinalEncoder()
    spec_df[spec_cols] = enc.fit_transform(spec_df[spec_cols])
    specs = (
        pd.DataFrame({"vehicle_id": vids})
          .merge(spec_df[["vehicle_id"] + spec_cols], on="vehicle_id")
    )[spec_cols].values.astype(int)

    joblib.dump(enc, ENCODER_PATH)
    if verbose:
        print(f"Saved encoder → {ENCODER_PATH}")

    return X, y, vids, specs


def save_to_h5(X, y, vids, specs, h5_path=H5_PATH):
    with h5py.File(h5_path, "w") as f:
        f.create_dataset("X_windows", data=X, compression="gzip")
        f.create_dataset("y_labels", data=y, compression="gzip")
        f.create_dataset("window_vids", data=vids, compression="gzip")
        f.create_dataset("specs_per_window", data=specs, compression="gzip")
    print(f"Saved H5 → {h5_path}")


def load_from_h5(h5_path=H5_PATH):
    with h5py.File(h5_path, "r") as f:
        X = f["X_windows"][:]
        y = f["y_labels"][:]
        vids = f["window_vids"][:]
        specs = f["specs_per_window"][:]
    return X, y, vids, specs


class RULCombinedDataset(Dataset):
    def __init__(self, X, specs, y):
        self.X = X
        self.specs = specs
        self.y = y.reshape(-1, 1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return (
            torch.from_numpy(self.specs[i]).long(),
            torch.from_numpy(self.X[i]).float(),
            torch.from_numpy(self.y[i]).float()
        )


def make_artifact_folder(model_name, suffix):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder = f"{model_name}-{suffix}-{ts}"
    path = os.path.join(ARTIFACT_ROOT, folder)
    os.makedirs(path, exist_ok=True)
    return path

# === Models ===
class TimeSeriesEmbedder(nn.Module):
    def __init__(self, num_features, d_model=128, n_heads=8, num_layers=2, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(num_features, d_model)
        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_heads, dropout=dropout, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)

    def forward(self, x):
        x = self.input_proj(x)
        x = self.encoder(x)
        return x[:, -1, :]


class CombinedRULModel(nn.Module):
    def __init__(self, num_sensor_features, context_length, categories, continuous_dim, cont_mean_std=None):
        super().__init__()
        self.tf = TimeSeriesEmbedder(num_sensor_features, continuous_dim)
        if cont_mean_std is None:
            cont_mean_std = torch.stack([torch.zeros(continuous_dim), torch.ones(continuous_dim)], dim=1)
        self.tab = TabTransformer(
            categories=categories,
            num_continuous=continuous_dim,
            dim=continuous_dim,
            dim_out=1,
            depth=6,
            heads=8,
            attn_dropout=0.1,
            ff_dropout=0.1,
            mlp_hidden_mults=(4,2),
            mlp_act=nn.ReLU(),
            continuous_mean_std=cont_mean_std
        )

    def forward(self, x_cat, x_ts):
        cont = self.tf(x_ts)
        return self.tab(x_cat, cont)



In [ ]:
# === DP Utilities for Spectral-DP ===
def spectral_dp_gradient_update(model, sigma, clip_bound):
    for name, param in model.named_parameters():
        if param.grad is None:
            continue
        grad = param.grad.detach()
        if grad.dim() == 2:
            U, S, Vh = torch.linalg.svd(grad, full_matrices=False)
            S_clipped = torch.clamp(S, max=clip_bound)
            noise = torch.randn_like(S_clipped) * sigma * clip_bound
            S_noisy = S_clipped + noise
            param.grad = (U @ torch.diag(S_noisy) @ Vh).to(param.grad.device)
        else:
            norm = torch.norm(grad)
            factor = min(1.0, clip_bound / (norm + 1e-6))
            grad_clipped = grad * factor
            noise = torch.randn_like(grad_clipped) * sigma * clip_bound
            param.grad = grad_clipped + noise



# # import torch

# def spectral_dp_gradient_update(model, sigma, clip_bound):
#     """
#     Performs a gradient update using the Spectral-DP method as described in
#     arXiv:2307.13231 for all model parameters.

#     This implementation reshapes non-2D gradients into 2D matrices to apply
#     the spectral perturbation, adhering to the paper's original proposal.

#     Args:
#         model (torch.nn.Module): The model to update.
#         sigma (float): The noise multiplier for differential privacy.
#         clip_bound (float): The clipping bound for the singular values.
#     """
#     for name, param in model.named_parameters():
#         if param.grad is None:
#             continue

#         grad = param.grad.detach()
#         original_shape = grad.shape

#         # --- Reshape non-2D gradients to 2D matrices ---
#         if grad.dim() > 2:
#             # For convolutional filters (e.g., 4D) or other high-dim tensors,
#             # flatten into a 2D matrix. A common way is (out_channels, -1).
#             grad_2d = grad.view(original_shape[0], -1)
#         elif grad.dim() == 1:
#             # For bias vectors (1D), reshape to a column vector (N, 1).
#             grad_2d = grad.view(-1, 1)
#         else:
#             # Gradient is already a 2D matrix.
#             grad_2d = grad

#         # --- Apply Spectral-DP to the 2D matrix ---
#         try:
#             # 1. Compute Singular Value Decomposition (SVD)
#             U, S, Vh = torch.linalg.svd(grad_2d, full_matrices=False)

#             # 2. Clip the singular values
#             S_clipped = torch.clamp(S, max=clip_bound)

#             # 3. Add noise to the clipped singular values
#             # The standard deviation is (sigma * clip_bound)
#             noise = torch.randn_like(S_clipped) * sigma * clip_bound
#             S_noisy = S_clipped + noise

#             # 4. Reconstruct the noisy 2D gradient
#             noisy_grad_2d = U @ torch.diag(S_noisy) @ Vh

#         except torch.linalg.LinAlgError:
#             # SVD can fail on some GPUs for ill-conditioned matrices.
#             # In this case, fall back to a standard norm clip and noise on the flattened grad.
#             # This is a practical fallback, though not explicitly detailed in the paper.
            
#             print(f"SVD failed for {name}. Using standard norm clipping as a fallback.")

            
#             norm = torch.norm(grad_2d)
#             factor = min(1.0, clip_bound / (norm + 1e-6))
#             grad_clipped = grad_2d * factor
#             noise = torch.randn_like(grad_clipped) * sigma * clip_bound
#             noisy_grad_2d = grad_clipped + noise


#         # --- Reshape the gradient back to its original shape and update ---
#         param.grad = noisy_grad_2d.reshape(original_shape).to(param.grad.device)







def get_criterion(): return MSELoss()
def get_optimizer(model,lr=1e-3): return Adam(model.parameters(),lr=lr)








# === Training function ===


def train(use_h5=False, pvt=False, sigma=0.1, clip_bound=1.0):# sigma=0.5
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BATCH_SIZE = 256
    NUM_EPOCHS = 50
    LR = 1e-3
    LR_PAT = 5
    ES_PAT = 11
    LR_F = 0.5

    # Load or create data
    if use_h5 and os.path.exists(H5_PATH):
        X, y, _, specs = load_from_h5()
    else:
        X, y, _, specs = create_X_y(sensor_features=None)
        save_to_h5(X, y, _, specs)

    encoder = joblib.load(ENCODER_PATH)
    cat_sizes = tuple(len(c) for c in encoder.categories_)

    Xtr, Xv, str_, sv, yr, yv = train_test_split(X, specs, y, test_size=0.2, random_state=42)
    tl = DataLoader(RULCombinedDataset(Xtr, str_, yr), batch_size=BATCH_SIZE, shuffle=True)
    vl = DataLoader(RULCombinedDataset(Xv, sv, yv), batch_size=BATCH_SIZE, shuffle=False)

    # Determine suffix based on DP flag
    suffix = "DP" if pvt else "NDP"

    # Artifacts
    art = make_artifact_folder("CombinedRULModel", suffix)
    logp = os.path.join(art, "train_val_log.txt")
    metap = os.path.join(art, "metadata.json")
    ckpt = os.path.join(art, "checkpoint.pth")

    # Metadata init
    meta = {
        "model_name": "CombinedRULModel",
        "num_sensor_features": X.shape[2],
        "context_length": X.shape[1],
        "continuous_dim": 128,
        "categories": list(cat_sizes),
        "batch_size": BATCH_SIZE,
        "learning_rate": LR,
        "num_epochs": NUM_EPOCHS,
        "privacy": "Spectral-DP" if pvt else "None",
        "dp_sigma": sigma if pvt else None,
        "dp_clip_bound": clip_bound if pvt else None
    }
    with open(metap, "w") as f:
        json.dump(meta, f, indent=4)

    # Model setup
    model = CombinedRULModel(X.shape[2], X.shape[1], cat_sizes, 128).to(DEVICE)
    crit = get_criterion()
    opt = get_optimizer(model, lr=LR)
    sched = StepLR(opt, step_size=1, gamma=LR_F)

    best = float('inf')
    noimp = 0

    with open(logp, "w") as f:
        f.write("epoch,train_loss,val_loss,epoch_time,lr,notes\n")

    start_all = time.perf_counter()
    for ep in range(1, NUM_EPOCHS + 1):
        ep_start = time.perf_counter()
        lr_cur = opt.param_groups[0]['lr']

        # Training loop
        model.train()
        tloss = 0
        for xc, xt, yb in tl:
            xc, xt, yb = xc.to(DEVICE), xt.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad()
            loss = crit(model(xc, xt), yb)
            loss.backward()
            if pvt:
                spectral_dp_gradient_update(model, sigma, clip_bound)
                opt.step()
            else:
                opt.step()
            tloss += loss.item() * yb.size(0)
        tloss /= len(tl.dataset)

        # Validation loop
        model.eval()
        vloss = 0
        with torch.no_grad():
            for xc, xt, yb in vl:
                xc, xt, yb = xc.to(DEVICE), xt.to(DEVICE), yb.to(DEVICE)
                l = crit(model(xc, xt), yb)
                vloss += l.item() * yb.size(0)
        vloss /= len(vl.dataset)

        # Checkpointing & scheduler
        elapsed = time.perf_counter() - ep_start
        h, m, s = map(int, [elapsed//3600, (elapsed%3600)//60, elapsed%60])
        et = f"{h:02d}:{m:02d}:{s:02d}"
        notes = ""
        if vloss < best:
            best = vloss
            noimp = 0
            torch.save(model.state_dict(), ckpt)
            notes = f"Saved at epoch {ep}"
        else:
            noimp += 1
            if noimp % LR_PAT == 0:
                sched.step()
                notes += " LR stepped"
            if noimp >= ES_PAT:
                notes += " Early stopping"

        with open(logp, "a") as f:
            f.write(f"{ep},{tloss:.6f},{vloss:.6f},{et},{lr_cur:.6g},{notes.strip()}\n")
        print(f"Epoch {ep:02d} Train {tloss:.4f} Val {vloss:.4f} Time {et} LR {lr_cur:.2e} {notes}")

        if noimp >= ES_PAT:
            print("Early stop")
            break

    total = time.perf_counter() - start_all
    h, m, s = map(int, [total//3600, (total%3600)//60, total%60])
    tt = f"{h:02d}:{m:02d}:{s:02d}"

    # Update metadata
    with open(metap, "r+") as f:
        d = json.load(f)
        d["total_training_time"] = tt
        f.seek(0)
        json.dump(d, f, indent=4)
        f.truncate()

    print(f"Done. Best val MSE {best:.4f}. Total time {tt}")

# === Entry Point ===
if __name__ == "__main__":
    # Set pvt=True to enable Spectral-DP
    train(use_h5=True, pvt=True, sigma=0.5)


In [2]:
import torch
torch.backends.cuda.preferred_linalg_library("magma")


<_LinalgBackend.Magma: 2>

# MATHEMATICAL LOGI

In [4]:
import torch

# Define a 4x4 matrix
A = torch.tensor([
    [3.0, 1.0, 1.0, 2.0],
    [4.0, 3.0, 1.0, 1.0],
    [1.0, 1.0, 3.0, 2.0],
    [2.0, 1.0, 2.0, 3.0],
    [1.0, 2.0, 3.0, 4.0]

])

# Perform SVD
U, S, Vh = torch.linalg.svd(A, full_matrices=False)

# Print components
print("U =", U)
print("S =", S)
print("Vh =", Vh)

# You can reconstruct A like this:
A_reconstructed = U @ torch.diag(S) @ Vh

U = tensor([[-0.3850, -0.2868,  0.6279,  0.1938],
        [-0.4614, -0.7425, -0.4439, -0.1275],
        [-0.3780,  0.3269, -0.4040,  0.7604],
        [-0.4445,  0.1387,  0.4663,  0.0595],
        [-0.5462,  0.4903, -0.1675, -0.6036]])
S = tensor([9.3481, 3.8020, 1.4381, 1.0437])
Vh = tensor([[-0.5150, -0.3941, -0.4823, -0.5890],
        [-0.7196, -0.2809,  0.4471,  0.4511],
        [ 0.3262, -0.6791, -0.4157,  0.5095],
        [ 0.3326, -0.5519,  0.6283, -0.4359]])


In [5]:
A = (U @ torch.diag(S) @ Vh)
print("Reconstructed A =", A_reconstructed)


Reconstructed A = tensor([[3.0000, 1.0000, 1.0000, 2.0000],
        [4.0000, 3.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 3.0000, 2.0000],
        [2.0000, 1.0000, 2.0000, 3.0000],
        [1.0000, 2.0000, 3.0000, 4.0000]])


In [6]:
import numpy as np

# Step 1: Define a small matrix A (2x2 for simplicity)
A = np.array([[4, 0],
              [3, -5]])

# Step 2: Compute AᵀA and AAᵀ
ATA = A.T @ A  # shape (2x2)
AAT = A @ A.T  # shape (2x2)

print("AᵀA:\n", ATA)
print("AAᵀ:\n", AAT)

# Step 3: Eigen decomposition of AᵀA to get V and S²
eigvals_V, V = np.linalg.eigh(ATA)
# Singular values = sqrt of eigenvalues
singular_values = np.sqrt(np.abs(eigvals_V[::-1]))  # sorted in decreasing order
V = V[:, ::-1]  # reverse to match sorted eigenvalues

print("Eigenvalues of AᵀA (S²):", eigvals_V[::-1])
print("Singular values (S):", singular_values)
print("Right singular vectors (V):\n", V)

# Step 4: Get U = (1/S) * A * V
S_inv = np.diag(1 / singular_values)
U = A @ V @ S_inv
print("Left singular vectors (U):\n", U)

# Step 5: Final SVD
S = np.diag(singular_values)
Vh = V.T

# Reconstruct A to verify
A_reconstructed = U @ S @ Vh
print("Reconstructed A:\n", A_reconstructed)

AᵀA:
 [[ 25 -15]
 [-15  25]]
AAᵀ:
 [[16 12]
 [12 34]]
Eigenvalues of AᵀA (S²): [40. 10.]
Singular values (S): [6.32455532 3.16227766]
Right singular vectors (V):
 [[-0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]]
Left singular vectors (U):
 [[-0.4472136  -0.89442719]
 [-0.89442719  0.4472136 ]]
Reconstructed A:
 [[ 4.  0.]
 [ 3. -5.]]
